In [1]:
!pip install webdriver_manager

In [2]:
!rm -rf /Users/briandekeijzer/.wdm
!pip install --upgrade webdriver_manager

In [3]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [4]:
!apt --fix-broken install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


카페 맘스홀릭

-> 카테고리 "임신&산후 우울증"

-> 제목에 "산후우울증" 단어를 포함

->댓글이 2개 이상인 글

-> 제목, 내용, 댓글1, 댓글2 컬럼으로 데이터프레임 생성

크롤링 크롬드라이버 자동설치 관련 오류 해결


https://nicedeveloper.tistory.com/entry/%ED%81%AC%EB%A1%AC%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B2%84-%EC%97%90%EB%9F%AC-service-option-browser-driver

In [5]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import pandas as pd
import os

# 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # 브라우저를 띄우지 않고 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--single-process')
chrome_options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 자동 설치 및 설정
#service = Service(ChromeDriverManager().install())

# Chrome WebDriver 시작
driver = webdriver.Chrome(options=chrome_options)

# 네이버 로그인
driver.get('https://nid.naver.com/nidlogin.login')
id_ = 'aim1016'
pw = ''

# Naver login 네이버 로그인
driver.execute_script("document.getElementsByName('id')[0].value=\'"+ id_ + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value=\'"+ pw + "\'")
driver.find_element(by=By.XPATH,value='//*[@id="log.login"]').click()
time.sleep(1)

# wanted naver cafe url, 수집할 페이지 및 게시글 수 설정
baseurl = 'https://cafe.naver.com/imsanbu/'
clubid = 10094499
menuid = 425
userDisplay = 50

pageNum = 1

title_df = []
url_df = []
data = []

for pageNum in range(1,30):
  url = baseurl + 'ArticleList.nhn?search.clubid=' + str(clubid) + '&search.menuid=' + str(menuid) +'&search.page='+ str(pageNum) +'&userDisplay=' + str(userDisplay)

  # 카페 내 게시글 목록 페이지 접속
  driver.get(url)
# driver.switch_to.frame('cafe_main')

  # BeautifulSoup을 사용하여 페이지 소스 파싱
  frame = driver.find_element(By.ID, 'cafe_main')
  driver.switch_to.frame(frame)
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  articles = soup.find_all('a', class_='article')

  for article in articles:
      title = article.text.strip()
      if '[산후우울증' not in title:
          continue
      title = title.replace("[산후우울증 후기]\n", "")
      title = title.replace("[산후우울증 문의]\n", "")
      title = title.replace("[산후우울증 정보]\n", "")
      title_df.append(title)

      article_url = 'https://cafe.naver.com' + article['href']
      url_df.append(article_url)
    # 글 내용 크롤링
   # driver.get(article_url)
   # article_soup = BeautifulSoup(driver.page_source, 'html.parser')
   # content = article_soup.find('div', class_='content CafeViewer').text.strip() if article_soup.find('div', class_='content CafeViewer') else "내용없음"

df= pd.DataFrame({'title':title_df, 'url':url_df})

In [6]:
df

,title,url
0,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
1,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
2,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
3,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
4,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
...,...,...
448,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
449,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
450,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...
451,\n ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...


## 깃헙 코드 응용

In [26]:
import pyperclip

def naver_login(driver, my_id, my_pw): # 네이버 로그인 함수
    # 설정
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # 브라우저를 띄우지 않고 실행
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--single-process')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)

    # 네이버 로그인
    driver.get("https://nid.naver.com/nidlogin.login")

    try:
        id_input = driver.find_element(By.CSS_SELECTOR, '#id')
        pw_input = driver.find_element(By.CSS_SELECTOR, '#pw')

        id_input.clear()
        pw_input.clear()

        id_input.send_keys(my_id)
        time.sleep(1)
        pw_input.send_keys(my_pw)
        time.sleep(1)

        driver.find_element(By.XPATH, '//*[@id="log.login"]').click()
        time.sleep(3)
    except Exception as e:
        print("Login failed:", e)

In [27]:
my_id = 'aim1016'
my_pw = ''

In [28]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pickle
from bs4 import BeautifulSoup
import pandas as pd

# wanted naver cafe url, 수집할 페이지 및 게시글 수 설정
baseurl = 'https://cafe.naver.com/imsanbu/'
clubid = 10094499
menuid = 425
userDisplay = 50

num_pages = 1

for pageNum in range(1, num_pages + 1):
    # 페이지 접근 전에 로그인
    naver_login(driver, my_id, my_pw)

    url = baseurl + f'ArticleList.nhn?search.clubid={clubid}&search.menuid={menuid}&search.page={pageNum}'
    driver.get(url)
    driver.switch_to.frame('cafe_main')

    articles = driver.find_elements(By.CSS_SELECTOR, 'a.article')
    for article in articles:
        try:
            article.click()
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[-1])

            post = {}
            try:
                post['title'] = driver.find_element(By.CSS_SELECTOR, 'span.title_text').text
            except:
                post['title'] = driver.find_element(By.CSS_SELECTOR, 'h3.title_text').text

            post['content'] = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text
            post['url'] = driver.current_url

            comments = []
            comment_elements = driver.find_elements(By.CSS_SELECTOR, 'div.comment_box')
            for comment_element in comment_elements:
                comment = {}
                comment['author'] = comment_element.find_element(By.CSS_SELECTOR, 'a.nickname').text
                comment['date'] = comment_element.find_element(By.CSS_SELECTOR, 'span.date').text
                comment['content'] = comment_element.find_element(By.CSS_SELECTOR, 'span.text_comment').text
                comments.append(comment)

            post['comments'] = comments
            data.append(post)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print("Failed to process article:", e)
            try:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            except:
                driver.quit()
                driver =  webdriver.Chrome(options=chrome_options)
                naver_login(driver, my_id, my_pw)
                driver.get(url)
                driver.switch_to.frame('cafe_main')
            continue

MaxRetryError: HTTPConnectionPool(host='localhost', port=48333): Max retries exceeded with url: /session/49bcb34f8cc200ddf6e91afd49a5e5ab/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7906b59ee9b0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70305&docId=470942608&enc=utf8&kinsrch_src=pc_tab_kin&qb=7IKw7ZuE7Jqw7Jq47Kad&rank=1&search_sort=0&section=kin.qna_ency_cafe&spq=0

In [30]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

def naver_login(driver, my_id, my_pw):
    # 네이버 로그인
    driver.get("https://nid.naver.com/nidlogin.login")

    try:
        id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#id'))
        )
        pw_input = driver.find_element(By.CSS_SELECTOR, '#pw')

        id_input.clear()
        pw_input.clear()

        id_input.send_keys(my_id)
        time.sleep(1)
        pw_input.send_keys(my_pw)
        time.sleep(1)

        driver.find_element(By.XPATH, '//*[@id="log.login"]').click()
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#log.logout'))
        )
    except Exception as e:
        print("Login failed:", e)


def crawl_naver_cafe(my_id, my_pw, num_pages):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--single-process')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)

    # wanted naver cafe url, 수집할 페이지 및 게시글 수 설정
    baseurl = 'https://cafe.naver.com/imsanbu/'
    clubid = 10094499
    menuid = 425
    userDisplay = 50

    data = []

    for pageNum in range(1, num_pages + 1):
        # 페이지 접근 전에 로그인
        naver_login(driver, my_id, my_pw)

        url = baseurl + f'ArticleList.nhn?search.clubid={clubid}&search.menuid={menuid}&search.page={pageNum}'
        driver.get(url)
        driver.switch_to.frame('cafe_main')

        articles = driver.find_elements(By.CSS_SELECTOR, 'a.article')
        for article in articles:
            try:
                article.click()
                WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                driver.switch_to.window(driver.window_handles[-1])

                post = {}
                try:
                    post['title'] = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'span.title_text'))
                    ).text
                except:
                    post['title'] = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'h3.title_text'))
                    ).text

                post['content'] = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.se-main-container'))
                ).text
                post['url'] = driver.current_url

                comments = []
                comment_elements = driver.find_elements(By.CSS_SELECTOR, 'div.comment_box')
                for comment_element in comment_elements:
                    comment = {}
                    comment['author'] = comment_element.find_element(By.CSS_SELECTOR, 'a.nickname').text
                    comment['date'] = comment_element.find_element(By.CSS_SELECTOR, 'span.date').text
                    comment['content'] = comment_element.find_element(By.CSS_SELECTOR, 'span.text_comment').text
                    comments.append(comment)

                post['comments'] = comments
                data.append(post)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            except Exception as e:
                print("Failed to process article:", e)
                try:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    driver.quit()
                    driver =  webdriver.Chrome(options=chrome_options)
                    naver_login(driver, my_id, my_pw)
                    driver.get(url)
                    driver.switch_to.frame('cafe_main')
                continue

    driver.quit()

    # 데이터 저장
    df = pd.DataFrame(data)
    df.to_csv('naver_cafe_data.csv', index=False, encoding='utf-8-sig')
    print("Data saved to naver_cafe_data.csv")

# 사용자 아이디와 비밀번호
my_id = 'aim1016'
my_pw = ''
num_pages = 1

# 크롤링 실행
crawl_naver_cafe(my_id, my_pw, num_pages)


Login failed: Message: 
Stacktrace:
#0 0x593cae84d6aa <unknown>
#1 0x593cae5300dc <unknown>
#2 0x593cae57c931 <unknown>
#3 0x593cae57ca21 <unknown>
#4 0x593cae5c1234 <unknown>
#5 0x593cae59f89d <unknown>
#6 0x593cae5be5c3 <unknown>
#7 0x593cae59f613 <unknown>
#8 0x593cae56f4f7 <unknown>
#9 0x593cae56fe4e <unknown>
#10 0x593cae81387b <unknown>
#11 0x593cae817921 <unknown>
#12 0x593cae7ff36e <unknown>
#13 0x593cae818482 <unknown>
#14 0x593cae7e3ccf <unknown>
#15 0x593cae83d0a8 <unknown>
#16 0x593cae83d280 <unknown>
#17 0x593cae84c7dc <unknown>
#18 0x7f0d6b797ac3 <unknown>

Failed to process article: Message: 
Stacktrace:
#0 0x593cae84d6aa <unknown>
#1 0x593cae5300dc <unknown>
#2 0x593cae57c931 <unknown>
#3 0x593cae57ca21 <unknown>
#4 0x593cae5c1234 <unknown>
#5 0x593cae59f89d <unknown>
#6 0x593cae5be5c3 <unknown>
#7 0x593cae59f613 <unknown>
#8 0x593cae56f4f7 <unknown>
#9 0x593cae56fe4e <unknown>
#10 0x593cae81387b <unknown>
#11 0x593cae817921 <unknown>
#12 0x593cae7ff36e <unknown>
#13 0x

Failed to process article: HTTPConnectionPool(host='localhost', port=52183): Max retries exceeded with url: /session/bc13218e90031ccfd7c9af8fcd597c23/element/f.2076F88B49DA813F0129C4A4C75540F8.d.68AEC676D1C0D27B3B4D0924A487CB7B.e.59/click (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7906e40823e0>: Failed to establish a new connection: [Errno 111] Connection refused'))


KeyboardInterrupt: 

In [36]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote
import time

# 위치 지정
#driver = webdriver.Chrome("d:/py_data/chromedriver.exe")
keys = Keys()
chrome_options = Options()
chrome_options.add_argument('--headless')  # 브라우저를 띄우지 않고 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--single-process')
chrome_options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 자동 설치 및 설정
#service = Service(ChromeDriverManager().install())

# Chrome WebDriver 시작
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://section.cafe.naver.com/")

keyword = input("찾으시려는 키워드를 입력하세요:")
END_PAGE = int(input("마지막 페이지를 입력하세요: ")) + 1
PAGE_url = "&p="

query_elm = driver.find_element(By.CSS_SELECTOR,"#header > div.snb_area > div > form > fieldset > div > div > input")
query_elm.click() # 검색창 클릭
query_elm.send_keys(keyword) #검색어 입력

# 엔터키 입력
query_elm.send_keys(keys.ENTER)
driver.set_page_load_timeout(30) # 로드 대기시간
driver.implicitly_wait(10)  # 브라우저에서 사용되는 엔진 자체에서 파싱되는 시간을 기다려 주는 메소드

# 전체글 클릭
driver.find_element_by_css_selector("#app > div > div.container > div > div.SectionSearchContent > div.section_search_tab_list > a:nth-child(3)").click()
url = driver.current_url
driver.implicitly_wait(10)

title_lst = []
doc_lst = []
comment_lst = []

for now_page in range(1,END_PAGE):
    driver.get(url+PAGE_url+str(now_page))
    time.sleep(2)     # 프로세스 자체를 지정한 시간동안 기다려 주는 역할(무조건 대기)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    item_lst = soup.find_all("div",class_="item_inner")

    for i in range(1,len(item_lst)+1):
        driver.find_element_by_css_selector("#app > div > div.container > div > div.SectionSearchContent > div.section_search_content > div > div.article_list_area > ul > li:nth-child("+str(i)+") > div > div > div > a").click()
        driver.implicitly_wait(10)
        driver.switch_to.window(driver.window_handles[-1])
        driver.switch_to_frame('cafe_main')
        driver.implicitly_wait(20)
        html = driver.page_source#소스 가져오기
        soup = bs(html, 'html.parser')


        try:
            title = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9 ]',"",soup.find('h3',class_='title_text').text)
        except:
            title=" "

        title_lst.append(title)

        try:
            doc_temp = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9 ]',"",soup.find("div",class_="se-module").text)
        except:
            doc_temp = " "
        doc_lst.append(doc_temp)

        comment_area = soup.find_all("div", class_="comment_area")

        comment = ""
        for i in comment_area:
            try:
                comment_temp = i.find("span",class_="text_comment").text
            except AttributeError:
                pass
            comment = comment + re.sub('[^a-zA-Zㄱ-ㅣ가-힣 ]', '',comment_temp)
        comment_lst.append(comment)

        driver.close()
        driver.switch_to.window(driver.window_handles[0]) # 맨 처음 탭으로 변경(0번 탭)
        driver.implicitly_wait(2)

driver.close()
df = pd.DataFrame({"title":title_lst, "doc":doc_lst, "comment":comment_lst})

찾으시려는 키워드를 입력하세요:산후우울증
마지막 페이지를 입력하세요: 1


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#header > div.snb_area > div > form > fieldset > div > div > input"}
  (Session info: chrome-headless-shell=126.0.6478.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55c9f92ba6aa <unknown>
#1 0x55c9f8f9d0dc <unknown>
#2 0x55c9f8fe9931 <unknown>
#3 0x55c9f8fe9a21 <unknown>
#4 0x55c9f902e234 <unknown>
#5 0x55c9f900c89d <unknown>
#6 0x55c9f902b5c3 <unknown>
#7 0x55c9f900c613 <unknown>
#8 0x55c9f8fdc4f7 <unknown>
#9 0x55c9f8fdce4e <unknown>
#10 0x55c9f928087b <unknown>
#11 0x55c9f9284921 <unknown>
#12 0x55c9f926c36e <unknown>
#13 0x55c9f9285482 <unknown>
#14 0x55c9f9250ccf <unknown>
#15 0x55c9f92aa0a8 <unknown>
#16 0x55c9f92aa280 <unknown>
#17 0x55c9f92b97dc <unknown>
#18 0x79250ef4bac3 <unknown>
